Marcin Wardyński  
wtorek, 9:45

## Laboratorium 7
### 7.4 AE

In [59]:
import importlib
import lab7_utils as utils
importlib.reload(utils)

import numpy as np
import pandas as pd
import tensorflow as tf

seed = 42

In [65]:
class SimpleAutoencoder(tf.keras.Model):
  def __init__(self, latent_dim, mid_dim):
    super(SimpleAutoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.mid_dim = mid_dim    
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(shape=(784,)),
            tf.keras.layers.Dense(mid_dim, activation=tf.nn.relu),
            tf.keras.layers.Dense(latent_dim, activation=tf.nn.relu),
        ]
    )
    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(shape=(latent_dim,)),
            tf.keras.layers.Dense(mid_dim, activation=tf.nn.relu),
            tf.keras.layers.Dense(units=28*28, activation=tf.nn.sigmoid),
        ]
    )

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded    

In [61]:
X_train, X_val, X_test, y_train, y_val, y_test = utils.get_dataset_for_ae(utils.Dataset_Select.MNIST.value, with_val=True)

In [95]:
autoencoder = SimpleAutoencoder(784, 196, 256)
autoencoder = tf.keras.Sequential([autoencoder.encoder, autoencoder.decoder])

In [96]:
import keras

autoencoder.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy)


In [98]:
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss=keras.losses.BinaryCrossentropy)

In [99]:
autoencoder.fit(X_train, X_train,
                epochs=10,
                shuffle=True,
                validation_data=(X_val, X_val))

Epoch 1/10
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 10.1245 - val_loss: 0.2633
Epoch 2/10
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.2638 - val_loss: 0.2631
Epoch 3/10
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.2636 - val_loss: 0.2631
Epoch 4/10
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.2638 - val_loss: 0.2629
Epoch 5/10
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.2643 - val_loss: 0.2630
Epoch 6/10
 151/1608 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.2632

KeyboardInterrupt: 

In [82]:
class SimpleAutoencoder(tf.keras.Model):
  def __init__(self, inout_dim, latent_dim, mid_dim):
    super(SimpleAutoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.mid_dim = mid_dim    
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(shape=(inout_dim, )),
            tf.keras.layers.Dense(mid_dim, activation=tf.nn.relu),
            tf.keras.layers.Dense(latent_dim, activation=tf.nn.relu),
        ]
    )
    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(shape=(latent_dim,)),
            tf.keras.layers.Dense(mid_dim, activation=tf.nn.relu),
            tf.keras.layers.Dense(units=inout_dim, activation=tf.nn.sigmoid),
        ]
    )

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded    

def create_simple_ae_as_sequence(inout_dim, latent_dim, mid_dim):
  model =  SimpleAutoencoder(inout_dim, latent_dim, mid_dim)
  return tf.keras.Sequential([model.encoder, model.decoder])

def create_simple_ae_from_params(params):
  return SimpleAutoencoder(params["model__inout_dim"], params["model__latent_dim"], params["model__mid_dim"])  

In [114]:
import keras

from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

def evaluate_autoencoder(dataset_name, params, create_plain_ae_fun, create_ae_from_params_fun):

    X_train, X_test, y_train, y_test = utils.get_dataset_for_ae(dataset_name=dataset_name, with_val=False)

    keras_reg = KerasRegressor(
        model=create_plain_ae_fun,
        loss="binary_crossentropy",
        optimizer="adam",
        optimizer__learning_rate=0.001,
        model__inout_dim=784,
        model__latent_dim=128,
        model__mid_dim=256,
        verbose=False,
    )

    grid_search = GridSearchCV(keras_reg, params, refit=False, cv=7, n_jobs=-1)
    grid_search.fit(X_train, X_train)

    print(grid_search.best_score_, grid_search.best_params_)

    X_train_s, X_val_s, _, _, _, _ = utils.get_dataset_for_ae(dataset_name=dataset_name, with_val=True)

    ae = create_ae_from_params_fun(grid_search.best_params_)
    ae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=grid_search.best_params_["optimizer__learning_rate"]), loss=keras.losses.BinaryCrossentropy)
    ae.fit(X_train_s, X_train_s,
                epochs=grid_search.best_params_["epochs"],
                shuffle=True,
                validation_data=(X_val_s, X_val_s))
    
    return ae
    
    

    

In [115]:
param_grid = {
    'model__inout_dim': [784],
    'model__latent_dim': [10, 64, 128, 192],
    'model__mid_dim': [256, 384, 512],
    'optimizer__learning_rate': [0.001, 0.005, 0.01],
    'epochs': [40]
}

ae = evaluate_autoencoder(utils.Dataset_Select.MNIST.value, param_grid, create_simple_ae_as_sequence, create_simple_ae_from_params)

/Users/mwardynski/Documents/ds/_semestr_9/uczenie_maszynowe/labs/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mwardynski/Documents/ds/_semestr_9/uczenie_maszynowe/labs/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mwardynski/Documents/ds/_semestr_9/uczenie_maszynowe/labs/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mwardynski/Documents/ds/_semestr_9/uczenie_maszynow

0.712966146204136 {'epochs': 40, 'model__inout_dim': 784, 'model__latent_dim': 192, 'model__mid_dim': 384, 'optimizer__learning_rate': 0.001}
Epoch 1/40
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.1542 - val_loss: 0.0820
Epoch 2/40
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0786 - val_loss: 0.0754
Epoch 3/40
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0735 - val_loss: 0.0734
Epoch 4/40
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0714 - val_loss: 0.0722
Epoch 5/40
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.0701 - val_loss: 0.0712
Epoch 6/40
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0695 - val_loss: 0.0698
Epoch 7/40
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0686 - val_loss: 0.0711
Epoch 8/40
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0681 - val_loss: 0.0694
Epoch 9/40
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0680 - val_loss: 0.0692
Epoch 10/40
1608/1608 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0

```
0.712966146204136 {'epochs': 40, 'model__inout_dim': 784, 'model__latent_dim': 192, 'model__mid_dim': 384, 'optimizer__learning_rate': 0.001}
```

In [119]:
ae_trained

<SimpleAutoencoder name=simple_autoencoder_88, built=True>

In [121]:
from sklearn.base import clone

frozen_encoder = ae_trained.encoder

for layer in frozen_encoder.layers:
    layer.trainable = False

In [127]:
from sklearn.base import BaseEstimator, TransformerMixin

class FrozenEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, encoder):
        self.encoder = encoder

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return self.encoder.predict(X)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

frozen_encoder = FrozenEncoder(encoder=ae_trained.encoder)

X_train, X_test, y_train, y_test = utils.get_dataset_for_ae(dataset_name=utils.Dataset_Select.MNIST.value, with_val=False)

pipeline_log_reg = Pipeline([
    ('frozen_encoder', frozen_encoder),
    ('log_reg', LogisticRegression(max_iter=2000, solver="newton-cg"))
])

pipeline_log_reg.fit(X_train, y_train)

y_pred = pipeline_log_reg.predict(X_test)

print(classification_report(y_test, y_pred))



1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 489us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       980
           1       0.98      0.98      0.98      1135
           2       0.92      0.91      0.92      1032
           3       0.89      0.91      0.90      1010
           4       0.92      0.93      0.93       982
           5       0.91      0.87      0.89       892
           6       0.95      0.96      0.95       958
           7       0.93      0.93      0.93      1028
           8       0.88      0.90      0.89       974
           9       0.91      0.90      0.91      1009

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000



In [ ]:
from sklearn.ensemble import RandomForestClassifier

pipeline_rand_frst = Pipeline([
    ('frozen_encoder', frozen_encoder),
    ('rand_frst', RandomForestClassifier(n_estimators=250, random_state=seed))
])

pipeline_rand_frst.fit(X_train, y_train)

y_pred = pipeline_rand_frst.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
ae = SimpleAutoencoder(784, 10, 128)



In [100]:
X_train_s, X_val_s, _, _, _, _ = utils.get_dataset_for_ae(dataset_name=utils.Dataset_Select.MNIST.value, with_val=True)

ae = SimpleAutoencoder(784, 128, 256)
ae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.05), loss=keras.losses.BinaryCrossentropy)
ae.fit(X_train_s, X_train_s,
            epochs=1,
            shuffle=True,
            validation_data=(X_val_s, X_val_s))

1608/1608 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: -40087309941364228096.0000 - val_loss: -907399326671443066880.0000


In [38]:
X_train, X_test, y_train, y_test = utils.get_dataset(dataset_name=utils.Dataset_Select.MNIST.value, print_sample_number=0)

In [16]:
from scikeras.wrappers import KerasClassifier

def create_ae(latent_dim, mid_dim):
    autoencoder = SimpleAutoencoder(latent_dim, mid_dim)
    return tf.keras.Sequential([autoencoder.encoder, autoencoder.decoder])

clf = KerasClassifier(
    model=create_ae,
    loss="binary_crossentropy",
    optimizer="adam",
    optimizer__learning_rate=0.1,
    model__latent_dim=(128,),
    model__mid_dim=(256,),
    verbose=False,
    epochs = 5,
)    

In [9]:
param_grid = {
    'model__latent_dim': [10, 128, 256],
    'model__mid_dim': [128, 256, 512],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'epochs': [1]
}

In [17]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=keras_clf, param_grid=param_grid, cv=7, verbose=1)
grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")

Fitting 7 folds for each of 27 candidates, totalling 189 fits


/Users/mwardynski/Documents/ds/_semestr_9/uczenie_maszynowe/labs/.venv/lib/python3.9/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwardynski/Documents/ds/_semestr_9/uczenie_maszynowe/labs/.venv/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
/Users/mwardynski/Documents/ds/_semestr_9/uczenie_maszynowe/labs/.venv/lib/python3.9/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwardynski/Documents/ds/_semestr_9/uczenie_maszynowe/labs/.venv/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` i

ValueError: 
All the 189 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
189 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/mwardynski/Documents/ds/_semestr_9/uczenie_maszynowe/labs/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/mwardynski/Documents/ds/_semestr_9/uczenie_maszynowe/labs/.venv/lib/python3.9/site-packages/scikeras/wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/Users/mwardynski/Documents/ds/_semestr_9/uczenie_maszynowe/labs/.venv/lib/python3.9/site-packages/scikeras/wrappers.py", line 770, in fit
    self._fit(
  File "/Users/mwardynski/Documents/ds/_seme

In [6]:
X_train, X_test, y_train, y_test = utils.get_dataset_for_ae(dataset_name=utils.Dataset_Select.MNIST.value, with_val=False)


In [10]:
X_train, X_test, y_train, y_test = utils.get_dataset(dataset_name=utils.Dataset_Select.MNIST.value, print_sample_number=0)

In [11]:
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow import keras

# Define the autoencoder creation function
def create_ae_new(inout_dim, latent_dim, mid_dim):
    autoencoder = SimpleAutoencoder(inout_dim, latent_dim, mid_dim)
    return tf.keras.Sequential([autoencoder.encoder, autoencoder.decoder])

clf = KerasClassifier(
    model=create_ae_new,
    loss="binary_crossentropy",
    optimizer="adam",
    optimizer__learning_rate=0.1,
    model__inout_dim=784,
    model__latent_dim=128,
    model__mid_dim=256,
    verbose=False,
    epochs = 5,
)

# Define parameter grid
param_grid = {
    'model__latent_dim': [128],
    'model__mid_dim': [256],
    "optimizer__learning_rate": [0.01],
}

from sklearn.model_selection import GridSearchCV

# Perform GridSearchCV
grid_search = GridSearchCV(clf, param_grid, refit=False, cv=3, scoring='accuracy')
grid_search.fit(X_train, X_train)

# Print the best parameters
print(f"Best parameters: {grid_search.best_params_}")


Best parameters: {'model__latent_dim': 128, 'model__mid_dim': 256, 'optimizer__learning_rate': 0.01}


In [12]:
print(grid_search.best_score_, grid_search.best_params_)

0.0001833333333333333 {'model__latent_dim': 128, 'model__mid_dim': 256, 'optimizer__learning_rate': 0.01}


In [20]:
import warnings
from tensorflow import get_logger
get_logger().setLevel('ERROR')
warnings.filterwarnings("ignore", message="Setting the random state for TF")

In [21]:
import numpy as np
from sklearn.datasets import make_classification


X, y = make_classification(1000, 20, n_informative=10, random_state=0)

X.shape, y.shape, y.mean()

((1000, 20), (1000,), np.float64(0.5))

In [22]:
import keras


def get_clf(meta, hidden_layer_sizes, dropout):
    n_features_in_ = meta["n_features_in_"]
    n_classes_ = meta["n_classes_"]
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=(n_features_in_,)))
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1, activation="sigmoid"))
    return model

In [26]:
from sklearn.model_selection import GridSearchCV


clf = KerasClassifier(
    model=get_clf,
    loss="binary_crossentropy",
    optimizer="adam",
    optimizer__learning_rate=0.1,
    model__hidden_layer_sizes=(100,),
    model__dropout=0.5,
    verbose=False,
)

In [28]:
params = {
    'optimizer__learning_rate': [0.05, 0.1],
    'model__hidden_layer_sizes': [(100, ), (50, 50, )],
    'model__dropout': [0, 0.5],
}

gs = GridSearchCV(clf, params, scoring='accuracy', n_jobs=-1, verbose=True)

gs.fit(X, y)

print(gs.best_score_, gs.best_params_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
0.7929999999999999 {'model__dropout': 0, 'model__hidden_layer_sizes': (100,), 'optimizer__learning_rate': 0.05}
